<a href="https://colab.research.google.com/github/shahabhi25/mynewrepo/blob/master/News_Article_popularity_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Model**

In [31]:
!pip3 install newspaper3k 
!pip install datefinder

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup
from newspaper import Article 
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
train_data = pd.read_csv('/content/drive/My Drive/NewsPopularity/news_popularity_training.csv')
test_data = pd.read_csv('/content/drive/My Drive/NewsPopularity/news_popularity_test.csv')

In [34]:
# Columns to drop

cols = np.asarray(['id', 'url', 'num_self_hrefs', 'kw_min_min', 
        'kw_max_min', 'kw_avg_min','kw_min_max', 'kw_max_max', 'kw_avg_max', 
        'kw_min_avg', 'kw_max_avg', 'kw_avg_avg', 'self_reference_min_shares', 
        'self_reference_max_shares', 'self_reference_avg_sharess', 'rate_positive_words',
        'rate_negative_words','abs_title_subjectivity','abs_title_sentiment_polarity'])
len(cols)

19

In [0]:
train_data.drop(cols, axis = 1, inplace = True)
test_data.drop(cols, axis = 1, inplace = True)

In [59]:
train_data.shape

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,popularity
0,651,8,467,0.540481,1.0,0.727273,4,1,1,4.563169,6,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0.410262,0.033384,0.156161,0.033466,0.366727,0.538728,0.171549,0.044968,0.004283,0.338528,0.033333,0.6,-0.450000,-0.800000,-0.100000,0.0,0.0,2
1,248,9,203,0.633508,1.0,0.688406,14,11,0,4.231527,9,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0.022223,0.368996,0.022222,0.564336,0.022223,0.472403,-0.061255,0.019704,0.024631,0.321591,0.136364,0.5,-0.304286,-0.500000,-0.071429,0.9,0.4,3
2,258,9,465,0.520788,1.0,0.703571,2,1,1,4.647312,6,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0.033334,0.033334,0.033368,0.033337,0.866627,0.510405,0.205746,0.051613,0.008602,0.443202,0.136364,1.0,-0.163690,-0.250000,-0.071429,0.0,0.0,2
3,247,9,618,0.575862,1.0,0.709677,14,0,11,4.645631,10,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0.420582,0.020216,0.020046,0.519155,0.020000,0.566894,0.178399,0.045307,0.011327,0.393220,0.050000,1.0,-0.340476,-0.666667,-0.050000,1.0,-1.0,2
4,93,10,394,0.532300,1.0,0.639676,7,2,1,4.565990,5,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0.640890,0.239056,0.040034,0.040020,0.040001,0.440037,0.144031,0.050761,0.015228,0.345152,0.033333,1.0,-0.358333,-0.800000,-0.100000,0.0,0.0,2


In [0]:
X = train_data.drop(labels=['popularity'], inplace=False, axis=1)
Y = train_data['popularity']-1

In [0]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [0]:
#Splitting the data 
X_train, X_test, Y_train, Y_test = train_test_split(X.values, Y.values, test_size = 0.1)

# Scaling the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [39]:
# XGBoost
from xgboost import XGBClassifier

"""
# Parameters Grid for Xgboost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='multi:softprob',
                    silent=True, nthread=1, )

random_search = RandomizedSearchCV(xgb, param_distributions=params, scoring='accuracy', n_jobs=-1, verbose=3, random_state=0)

random_search.fit(X_train, Y_train)

print(random_search.best_params_)
"""

"\n# Parameters Grid for Xgboost\nparams = {\n        'min_child_weight': [1, 5, 10],\n        'gamma': [0.5, 1, 1.5, 2, 5],\n        'subsample': [0.6, 0.8, 1.0],\n        'colsample_bytree': [0.6, 0.8, 1.0],\n        'max_depth': [3, 4, 5]\n        }\n\nxgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='multi:softprob',\n                    silent=True, nthread=1, )\n\nrandom_search = RandomizedSearchCV(xgb, param_distributions=params, scoring='accuracy', n_jobs=-1, verbose=3, random_state=0)\n\nrandom_search.fit(X_train, Y_train)\n\nprint(random_search.best_params_)\n"

In [40]:
clf = XGBClassifier(subsample= 1.0, min_child_weight=10, max_depth= 5, gamma=5, colsample_bytree=0.6)

clf.fit(X_train, Y_train)


print(clf.score(X_train, Y_train))
print(clf.score(X_test, Y_test))

0.5392222222222223
0.5103333333333333


# **Crawling News**

In [0]:
url = "https://timesofindia.indiatimes.com/world"
r = requests.get(url)

In [0]:
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find_all('a', attrs = {'class':'w_img'})

In [0]:
news=[]
for row in table: 
    if not row['href'].startswith('http'):
        news.append('https://timesofindia.indiatimes.com'+row['href'])

In [0]:
df=[]
for i in news:
    article = Article(i, language="en")
    article.download() 
    article.parse() 
    article.nlp() 
    data={}
    data['Title']=article.title
    data['Text']=article.text
    data['Summary']=article.summary
    data['Keywords']=article.keywords
    df.append(data)

In [45]:
dataset=pd.DataFrame(df)
dataset.head()

,Title,Text,Summary,Keywords
0,France Lockdown Lift: France to outline plans ...,Nurse Sandrine poses as she works in a hotel o...,"Restaurants, cafes and cinemas will have to re...","[number, france, coronavirus, told, times, loc..."
1,Covid-19: Testing won't 'be a problem' for reo...,"Apr 28, 2020, 08:40AM IST\n\nSource: AP\n\nThe...","Apr 28, 2020, 08:40AM ISTSource: APThe White H...","[wont, president, covid19, virus, white, house..."
2,Covid-19: Some Georgia restaurants reopen for ...,"Apr 28, 2020, 08:43AM IST\n\nSource: AP\n\nWit...","Apr 28, 2020, 08:43AM ISTSource: APWith tables...","[customers, dinein, covid19, wearing, service,..."
3,'Very good idea' on Kim Jong Un's health: Pres...,"Apr 28, 2020, 08:39AM IST\n\nSource: AP\n\nNor...","Apr 28, 2020, 08:39AM ISTSource: APNorth Korea...","[president, jong, rumors, good, speaking, futu..."
4,Covid-19: Mayor entertains residents after imp...,"Apr 27, 2020, 08:27AM IST\n\nSource: AP\n\nThe...","Apr 27, 2020, 08:27AM ISTSource: APThe videos,...","[try, residents, kawakami, dont, way, entertai..."


In [46]:
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from textblob import TextBlob
nltk.download('stopwords')

stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
def rate_unique(words):
    words = word_tokenize(words)
    no_order = list(set(words))
    rate_unique = len(no_order)/len(words)
    return rate_unique

In [0]:
def rate_nonstop(words):
    words = word_tokenize(words)
    filtered_sentence = [w for w in words if not w in stopwords]
    rate_nonstop = len(filtered_sentence)/len(words)
    no_order = list(set(filtered_sentence))
    rate_unique_nonstop = len(no_order)/len(words)
    return rate_nonstop,rate_unique_nonstop

In [0]:
def avg_token(words):
    words = word_tokenize(words)
    length=[]
    for i in words:
        length.append(len(i))
    return np.average(length)

In [0]:
import datefinder
import datetime 
from datetime import date 

def day(article_text):
    article = article_text
    if len(list(datefinder.find_dates(article)))>0:
        date=str(list(datefinder.find_dates(article))[0])
        date=date.split()
        date=date[0]
        year, month, day = date.split('-')     
        day_name = datetime.date(int(year), int(month), int(day)) 
        return day_name.strftime("%A")
    return "Monday"

def time_delta(article_text):
  article = article_text
  if len(list(datefinder.find_dates(article)))>0:
      date=list(datefinder.find_dates(article))[0]
      ini_time_for_now = datetime.datetime.today()
      return int((ini_time_for_now-date).days)
  return 0

In [0]:
pos_words=[]
neg_words=[]
def polar(words):
    all_tokens = word_tokenize(words)
    for i in all_tokens:
        analysis=TextBlob(i)
        polarity=analysis.sentiment.polarity
        if polarity>0:
            pos_words.append(i)
        if polarity<0:
            neg_words.append(i)
    return pos_words,neg_words

In [0]:
def rates(words):
    words=polar(words)
    pos=words[0]
    neg=words[1]
    all_words=words
    global_rate_positive_words=(len(pos)/len(all_words))/100
    global_rate_negative_words=(len(neg)/len(all_words))/100
    pol_pos=[]
    pol_neg=[]
    for i in pos:
        analysis=TextBlob(i)
        pol_pos.append(analysis.sentiment.polarity)
        avg_positive_polarity=analysis.sentiment.polarity
    for j in neg:
        analysis2=TextBlob(j)
        pol_neg.append(analysis2.sentiment.polarity)
        avg_negative_polarity=analysis2.sentiment.polarity
    min_positive_polarity=min(pol_pos)
    max_positive_polarity=max(pol_pos)
    min_negative_polarity=min(pol_neg)
    max_negative_polarity=max(pol_neg)
    avg_positive_polarity=np.average(pol_pos)
    avg_negative_polarity=np.average(pol_neg)
    return global_rate_positive_words,global_rate_negative_words,avg_positive_polarity,min_positive_polarity, max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import gensim, spacy

n_features = 1000
n_components = 5
n_top_words = 20

data_samples = dataset['Text']

nlp = spacy.load('en', disable=['parser', 'ner'])

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out



tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')

data_vectorized = tf_vectorizer.fit_transform(lemmatization(sent_to_words(data_samples)))

lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

topic_prob_score =lda.fit_transform(data_vectorized)


0.0046599979900204015

In [64]:
df2=[]
for i,art in enumerate(news):
    pred_info={}
    article = Article(art, language="en") # en for English 
    article.download() 
    article.parse()
    analysis=TextBlob(article.text)
    polarity=analysis.sentiment.polarity
    title_analysis=TextBlob(article.title)
    pred_info['text']=article.text
    pred_info['timedelta'] = time_delta(article.text)
    pred_info['n_tokens_title']=len(word_tokenize(article.title))
    pred_info['n_tokens_content']=len(word_tokenize(article.text))
    pred_info['n_unique_tokens']=rate_unique(article.text)
    pred_info['n_non_stop_words']=rate_nonstop(article.text)[0]
    pred_info['n_non_stop_unique_tokens']=rate_nonstop(article.text)[1]
    pred_info['num_hrefs']=article.html.count("https://timesofindia.indiatimes.com")
    pred_info['num_imgs']=len(article.images)
    pred_info['num_videos']=len(article.movies)
    pred_info['average_token_length']=avg_token(article.text)
    pred_info['num_keywords']=len(article.keywords)
    
    if "life-style" in article.url:
        pred_info['data_channel_is_lifestyle']=1
    else:
        pred_info['data_channel_is_lifestyle']=0
    if "etimes" in article.url:
        pred_info['data_channel_is_entertainment']=1
    else:
        pred_info['data_channel_is_entertainment']=0
    if "business" in article.url:
        pred_info['data_channel_is_bus']=1
    else:
        pred_info['data_channel_is_bus']=0
    if "social media" or "facebook" or "whatsapp" in article.text.lower():
        data_channel_is_socmed=1
        data_channel_is_tech=0
        data_channel_is_world=0
    else:
        data_channel_is_socmed=0
    if ("technology" or "tech" in article.text.lower()) or ("technology" or "tech" in article.url):
        data_channel_is_tech=1
        data_channel_is_socmed=0
        data_channel_is_world=0
    else:
        data_channel_is_tech=0
    if "world" in article.url:
        data_channel_is_world=1
        data_channel_is_tech=0
        data_channel_is_socmed=0
    else:
        data_channel_is_world=0
        
    pred_info['data_channel_is_socmed']=data_channel_is_socmed
    pred_info['data_channel_is_tech']=data_channel_is_tech
    pred_info['data_channel_is_world']=data_channel_is_world
    
    if day(art)=="Monday":
        pred_info['weekday_is_monday']=1
    else:
        pred_info['weekday_is_monday']=0
    if day(art)=="Tuesday":
        pred_info['weekday_is_tuesday']=1
    else:
        pred_info['weekday_is_tuesday']=0
    if day(art)=="Wednesday":
        pred_info['weekday_is_wednesday']=1
    else:
        pred_info['weekday_is_wednesday']=0
    if day(art)=="Thursday":
        pred_info['weekday_is_thursday']=1
    else:
        pred_info['weekday_is_thursday']=0
    if day(art)=="Friday":
        pred_info['weekday_is_friday']=1
    else:
        pred_info['weekday_is_friday']=0
    if day(art)=="Saturday":
        pred_info['weekday_is_saturday']=1
        pred_info['is_weekend']=1
    else:
        pred_info['weekday_is_saturday']=0
    if day(art)=="Sunday":
        pred_info['weekday_is_sunday']=1
        pred_info['is_weekend']=1
    else:
        pred_info['weekday_is_sunday']=0
        pred_info['is_weekend']=0
        
    pred_info['LDA_00'] = topic_prob_score[i,0]
    pred_info['LDA_01'] = topic_prob_score[i,1]
    pred_info['LDA_02'] = topic_prob_score[i,2]
    pred_info['LDA_03'] = topic_prob_score[i,3]
    pred_info['LDA_04'] = topic_prob_score[i,4]
    pred_info['global_subjectivity']=analysis.sentiment.subjectivity
    pred_info['global_sentiment_polarity']=analysis.sentiment.polarity
    pred_info['global_rate_positive_words']=rates(article.text)[0]
    pred_info['global_rate_negative_words']=rates(article.text)[1]
    pred_info['avg_positive_polarity']=rates(article.text)[2]
    pred_info['min_positive_polarity']=rates(article.text)[3]
    pred_info['max_positive_polarity']=rates(article.text)[4]
    pred_info['avg_negative_polarity']=rates(article.text)[5]
    pred_info['min_negative_polarity']=rates(article.text)[6]
    pred_info['max_negative_polarity']=rates(article.text)[7]    
    pred_info['title_subjectivity']=title_analysis.sentiment.subjectivity
    pred_info['title_sentiment_polarity']=title_analysis.sentiment.polarity
    df2.append(pred_info)

/usr/local/lib/python3.6/dist-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname IST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [66]:
pred_df=pd.DataFrame(df2)
#pred_test=pred_df.drop(['text'],axis=1)
#pred_test = pred_test.join(ndf)
pred_test.head()

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity
0,0,12,559,0.583184,0.688730,0.508050,230,13,0,4.737030,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0.001305,0.994780,0.001310,0.001301,0.001304,0.426497,0.122980,11.990,4.87,0.347161,0.033333,1.0,-0.247088,-1.0,-0.0125,0.00,0.00
1,0,15,152,0.736842,0.684211,0.539474,185,9,0,4.486842,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0.004642,0.004660,0.981454,0.004623,0.004621,0.342472,0.174148,12.800,4.98,0.346377,0.033333,1.0,-0.248435,-1.0,-0.0125,0.00,0.00
2,0,9,144,0.750000,0.708333,0.569444,185,9,0,4.472222,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0.004151,0.983380,0.004170,0.004150,0.004150,0.398452,0.010357,13.095,5.10,0.347580,0.033333,1.0,-0.247133,-1.0,-0.0125,0.00,0.00
3,0,14,181,0.662983,0.690608,0.491713,185,9,0,4.000000,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0.006151,0.006223,0.975306,0.006150,0.006170,0.287167,0.074750,13.215,5.30,0.348489,0.033333,1.0,-0.246887,-1.0,-0.0125,0.78,0.91
4,1,8,190,0.594737,0.589474,0.431579,185,9,0,3.726316,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0.005091,0.005151,0.979561,0.005089,0.005108,0.643750,0.300000,13.350,5.36,0.349216,0.033333,1.0,-0.246922,-1.0,-0.0125,0.00,0.00


In [70]:
pred_test.columns==X.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [71]:
test2=pd.DataFrame(clf.predict(pred_test.values),pred_df['text'])
test2.reset_index(level=0, inplace=True)
test2 = test2.rename(index=str, columns={"index": "News", 0: "Virality"})
test2

,text,Virality
0,Nurse Sandrine poses as she works in a hotel o...,1
1,"Apr 28, 2020, 08:40AM IST\n\nSource: AP\n\nThe...",1
2,"Apr 28, 2020, 08:43AM IST\n\nSource: AP\n\nWit...",1
3,"Apr 28, 2020, 08:39AM IST\n\nSource: AP\n\nNor...",1
4,"Apr 27, 2020, 08:27AM IST\n\nSource: AP\n\nThe...",0
5,"Apr 27, 2020, 08:40AM IST\n\nSource: Times Now...",1
6,"Apr 27, 2020, 08:28AM IST\n\nSource: AP\n\nPol...",1
7,"Apr 22, 2020, 04:03PM IST\n\nSource: AP\n\nInd...",2
8,"Apr 22, 2020, 03:58PM IST\n\nSource: AP\n\nIra...",0
9,"Apr 22, 2020, 04:04PM IST\n\nSource: AP\n\nAn ...",2
